In [12]:
NIR3_img_url = r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/2020-05-07-20-31-25_38/ms_2_images/NIR3/frames/image_2894.tiff'
NIR2_img_url = r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/2020-05-07-20-31-25_38/ms_2_images/NIR2/frames/image_2894.tiff'
NIR1_img_url = r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/2020-05-07-20-31-25_38/ms_2_images/NIR1/frames/image_2894.tiff'
color_img_url = r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/2020-05-07-20-31-25_38/video_color_images/frames/image_8739.jpg'


In [15]:
import smbclient
import numpy as np
import cv2

smbclient.ClientConfig(username=username, password=password)

# Optional - register the credentials with a server (overrides ClientConfig for that server)
smbclient.register_session("192.168.152.34", username=username, password=password)

channel_urls = [NIR1_img_url, NIR2_img_url, NIR3_img_url]

three_channel = []
for url in channel_urls:
    with smbclient.open_file(url, 'rb') as f:
        arr = np.asarray(bytearray(f.read()), dtype=np.uint8)
        image = cv2.imdecode(arr, -1)
        im = np.asarray(image)
        print(im.shape)
        (h, w) = image.shape[:2]
        image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)
        three_channel.append(image)
        
print(np.asarray(three_channel).shape)
print(np.stack(three_channel, axis=2).shape)

(341, 340)
(341, 340)
(341, 340)
(3, 256, 256)
(256, 256, 3)


In [13]:
with smbclient.open_file(color_img_url, 'rb') as f:
    arr = np.asarray(bytearray(f.read()), dtype=np.uint8)
    image = cv2.imdecode(arr, -1)
    im = np.asarray(image)
    print(im.shape)
    (h, w) = image.shape[:2]
    image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)

(480, 640, 3)


In [46]:
import smbclient
import json
import cv2
from pathlib import Path
import numpy as np
import zipfile

def write_to_YOLO(dataset_folder, labels_folder, img_url, val=False, test=False):
    img_name = Path(img_url).name
    channel_urls = [img_url, img_url.replace('NIR1', 'NIR2'), img_url.replace('NIR1', 'NIR3')]
    three_channel = []
    
    for url in channel_urls:
        with smbclient.open_file(img_url, 'rb') as f:
            arr = np.asarray(bytearray(f.read()), dtype=np.uint8)
            image = cv2.imdecode(arr, -1)
            (h, w) = image.shape[:2]
            image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)
            three_channel.append(image)
        
    image = np.stack(three_channel, axis=2)
    
    if val:
        with open(labels_folder + img_name.replace('.tiff', '.txt'), 'r') as fd:
            data = fd.read()
            x,y,width,height = [float(i) for i in data.strip('\n').strip(' ').split(' ') if i.isdigit()]

            x_center = (x + (width / 2 )) / w
            y_center = (y + (height / 2)) / h
            width = width / w
            height = height / h
            klass = 0
            with open(dataset_folder + '/labels/val/' + img_name.replace('.tiff', '.txt'), 'w+') as f:
                f.write(' '.join([str(i) for i in [klass, x_center, y_center, width, height]]))
        # save the images
        cv2.imwrite(dataset_folder + '/images/val/' + img_name, image)

    else:
        with open(labels_folder + img_name.replace('.tiff', '.txt'), 'r') as fd:
            data = fd.read()
            x,y,width,height = [float(i) for i in data.strip('\n').strip(' ').split(' ') if i.isdigit()]

            x_center = (x + (width / 2 )) / w
            y_center = (y + (height / 2)) / h
            width = width / w
            height = height / h
            klass = 0
            with open(dataset_folder + '/labels/train/' + img_name.replace('.tiff', '.txt'), 'w+') as f:
                f.write(' '.join([str(i) for i in [klass, x_center, y_center, width, height]]))
        # save the images
        cv2.imwrite(dataset_folder + '/images/train/' + img_name, image)
        

def mount_smb(username, password, smb_paths, train_dataset_folder, val_dataset_folder, train_labels_folder, val_labels_folder, train_imgs, val_imgs):
    # Optional - specify the default credentials to use on the global config object
    smbclient.ClientConfig(username=username, password=password)

    # Optional - register the credentials with a server (overrides ClientConfig for that server)
    smbclient.register_session("192.168.152.34", username=username, password=password)
            
    train_count = 0
    val_count = 0
    for root, dirs, files in smbclient.walk(r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/', 'r'):
        break
    for di in dirs:
        for _, _, files in smbclient.walk(r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/' + di + smb_path):
            break
        for file in files:
            if file.replace('.tiff', '.txt') in train_imgs:
                train_count += 1
                write_to_YOLO(train_dataset_folder, train_labels_folder, r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/' + di + smb_path + file)
                
    for root, dirs, files in smbclient.walk(r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/06_03_20/Yuneec_Typhoon/2020-05-07-20-38-19/', 'r'):
        break
    for di in dirs:
        for _, _, files in smbclient.walk(r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/06_03_20/Yuneec_Typhoon/2020-05-07-20-38-19/' + di + smb_path):
            break
        for file in files:
            if file.replace('.tiff', '.txt') in val_imgs:
                val_count += 1
                write_to_YOLO(val_dataset_folder, val_labels_folder, r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/06_03_20/Yuneec_Typhoon/2020-05-07-20-38-19/' + di + smb_path + file, val=True)
                
    print("{} training images".format(train_count))
    print("{} validation images".format(val_count))

In [47]:
import glob
train_files = [Path(p).name for p in glob.glob('labels/labels_05_13_20_multispectral/*.txt')]
val_files = [Path(p).name for p in glob.glob('labels/labels_06_03_20_multispectral/*.txt')]
print('training files: {}'.format(len(train_files)))
print('validation files: {}'.format(len(val_files)))

smb_path = '/ms_2_images/NIR1/frames/'
train_dataset_folder = 'yolov3/labels_05_13_20_NIR'
val_dataset_folder = 'yolov3/labels_06_03_20_NIR'
train_labels_folder = 'labels/labels_05_13_20_multispectral/'
val_labels_folder = 'labels/labels_06_03_20_multispectral/'
mount_smb(username, password, smb_path, train_dataset_folder, val_dataset_folder, train_labels_folder, val_labels_folder, train_files, val_files)


training files: 1951
validation files: 431
1951 training images
431 validation images


In [32]:
!python train.py --img 256 --batch 32 --epochs 20 --weights yolov3.pt --data NIR.yaml  --nosave --workers 4

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/utils/_process_posix.py", line 157, in system
    child = pexpect.spawn(self.sh, args=['-c', cmd])  # Vanilla Pexpect
  File "/opt/conda/lib/python3.8/site-packages/pexpect/pty_spawn.py", line 205, in __init__
    self._spawn(command, args, preexec_fn, dimensions)
  File "/opt/conda/lib/python3.8/site-packages/pexpect/pty_spawn.py", line 303, in _spawn
    self.ptyproc = self._spawnpty(self.args, env=self.env,
  File "/opt/conda/lib/python3.8/site-packages/pexpect/pty_spawn.py", line 315, in _spawnpty
    return ptyprocess.PtyProcess.spawn(args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/ptyprocess/ptyprocess.py", line 315, in spawn
    exec_err_data = os.read(exec_err_pipe_read, 4096)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.p

TypeError: object of type 'NoneType' has no len()